In [1]:
import mne, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io
import torch
from sklearn.preprocessing import StandardScaler
import scipy.signal as signal

In [2]:
def preprocess_eeg(eeg_data, fs):
    # Apply common average reference (CAR) across all channels
    average_reference = np.mean(eeg_data, axis=0)
    car_eeg = eeg_data - average_reference

    # Apply filter between 0.15 Hz and 40 Hz to the CAR-corrected EEG data
    b, a = signal.butter(4, [0.15, 40], 'bandpass', fs=fs)
    filtered_eeg = signal.filtfilt(b, a, car_eeg, axis=1)

    return filtered_eeg

def read_and_preprocess_seed_iv(file_path):
    mat = scipy.io.loadmat(file_path)
    eeg_keys = [key for key in mat.keys() if 'eeg' in key]
    
    if not eeg_keys:
        raise KeyError("No keys containing 'eeg' found in the .mat file.")
    
    eeg_key = eeg_keys[0]
    raw_eeg = mat[eeg_key]
    
    fs = 200  # Sampling frequency
    preprocessed_eeg = preprocess_eeg(raw_eeg, fs)
    
    return preprocessed_eeg, fs

def create_raw_eeg_object(data, sfreq):
    # Define the channel names
    ch_names = ['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 
                'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 
                'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 
                'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 
                'PO4', 'PO6', 'PO8', 'CB1', 'O1', 'OZ', 'O2', 'CB2']

    ch_types = ['eeg'] * len(ch_names)
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
    
    # Create the MNE Raw object
    raw = mne.io.RawArray(data / 1e6, info)
    
    return raw




In [3]:
def convert2edf(dataset_folder, file_name):
    #input the EEG origin data
    path = os.path.join(dataset_folder, file_name)
    file_path = path
    preprocessed_data, sfreq = read_and_preprocess_seed_iv(file_path)
    raw_eeg = create_raw_eeg_object(preprocessed_data, sfreq)
    
    # Optionally plot the raw EEG data
    #raw_eeg.plot(show_scrollbars=True, show_scalebars=True, n_channels=raw_eeg.info['nchan'])

    #add montage
    montage = mne.channels.make_standard_montage('standard_1005')
    raw_eeg.set_montage(montage, match_case=False)
    
    # Save as EDF
    edf_file_path = file_name+'.edf'
    raw_eeg.export(edf_file_path, fmt='edf', overwrite=True)
    
    # Save as FIF
    fif_file_path = file_name+'.fif'
    raw_eeg.save(fif_file_path, overwrite=True)
    
    print(f"Data saved as EDF at: {edf_file_path}")
    print(f"Data saved as FIF at: {fif_file_path}")

In [4]:

dataset_folder = r"F:\ML\Multi\EEG-DomainAdaptation\dataset\SEED_IV\eeg_raw_data\1"
file_name = "1_20160518.mat"
convert2edf(dataset_folder, file_name)

Creating RawArray with float64 data, n_channels=62, n_times=33601
    Range : 0 ... 33600 =      0.000 ...   168.000 secs
Ready.


ValueError: DigMontage is only a subset of info. There are 2 channel positions not present in the DigMontage. The channels missing from the montage are:

['CB1', 'CB2'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.